## Demo (Neuromeka TVE) - White board sweeping

### Prepare the task scene
* Prepare 2 Indy7, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [1]:
import os
import sys
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.utils.code_scraps import add_indy_sweep_tool, \
        use_current_place_point_only, use_current_sub_binders_only, \
        finish_L_shape, set_l_shape_object, ForceModeSwitcher, double_sweep_motions

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
INDY_IP = '192.168.21.6'
PANDA_ROBOT_IP = '192.168.21.13'
PANDA_REPEATER_IP = '192.168.21.14'

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection command:
indy0: False
panda1: False


# Init camera

In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
_ = stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
# deprecated: s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene
* **[IMPORTANT]** Make sure robot is visible, or correct saved file is used

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment
* **[IMPORTANT]** Make sure environment items are visible, or correct saved file is used

In [7]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

# add cam poles
ptems = s_builder.add_poles({"cam0": stereo.ref_coord_inv[:3,3], "cam1":np.matmul(stereo.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.NAME_DICT["floor"].dims = (2,1,0.01)
gscene.set_workspace_boundary( -0.8, 0.8, -0.5, 0.5, -0.1, 1.1)

Please create a subscriber to the marker


In [8]:
gscene.update_markers_all()

## add environmental components

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

add_indy_sweep_tool(gscene, "indy0", face_name="brush_face", tool_offset=0.14)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, 
                     point=(0,0,gscene.NAME_DICT['brush_face'].dims[2]/2), rpy=(0,np.pi,0))

track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.1, track.dims[1]-0.15, track.dims[2])
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane, point=None)
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

gscene.update_markers_all()

TOOL_DIM = [0.06, 0.10]
TACK_WIDTH = 0.16
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
WP_REF_M = np.array([0, WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
FC_THIC = 2e-3
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    face = gscene.create_safe(GEOTYPE.BOX, "face{}".format(i_trc+1), "base_link", 
                              (TRACK_DIM[0] - TOOL_DIM[0]/2,TACK_WIDTH,FC_THIC), 
                             center=tuple(WP_REF_M+[0,TRACK_STEP*i_trc])+(TRACK_DIM[2]/2 + FC_THIC/2,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, face))
    
    gscene.update_markers_all()

sweep_list = []
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, face = track_tem
    sweep_ = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                   action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])}, 
                                   clearance=[face])
    sweep_list.append(sweep_)
    


## add movable objects

In [10]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
gscene.update_markers_all()
    
for gname in sorted(gtem_dict.keys()):
    pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)

# ## add blocking structure on table
# gscene.create_safe(gtype=GEOTYPE.BOX, name="blocker", link_name="base_link",
#                             center=(-0.27,-0.2,0.15), dims=(0.1,0.3, 0.4), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
#                             collision=True, fixed=True)

### planners

In [11]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

In [12]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
tcheck = TaskClearanceChecker(pscene, gcheck)
checkers_all = [tcheck, rcheck, gcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)
mplan.motion_filters = checkers_all
gscene.show_pose(crob.home_pose)

from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)

# remove place points except for the current one
use_current_place_point_only(pscene, initial_state)

gscene.update_markers_all()
print("subjects: {}".format(pscene.subject_name_list))
print("initial node: {}".format(initial_state.node))

subjects: ['box1', 'sweep1', 'sweep2', 'sweep3']
initial node: ('track_face', 0, 0, 0)


# CustomRule

In [13]:
urules = [SweepLineTask.make_unstopable_rule(pscene, [sname]) for sname in ["sweep1", "sweep2", "sweep3"]]
urule_tot = lambda pscene, node, leaf: all([urule(pscene, node, leaf) for urule in urules]) 
tplan.explicit_rule = urule_tot
tplan.explicit_edges = {node : [node] for node in [node for node in tplan.node_list if 1 not in node]}
mplan.incremental_constraint_motion = True

## Detect once

In [28]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)
            
    gscene.clear_non_fixed()
    
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
    
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()

## Plan & show

In [29]:
from pkg.planning.motion.moveit.moveit_py import PlannerConfig
VEL = 0.5
ACC = 0.3

In [30]:
mplan.reset_log(False)
gtimer.reset()
print(initial_state.node)

obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    ppline.search(from_state, goal_nodes, verbose=False, display=False, dt_vis=0.01, 
                  timeout_loop=100, multiprocess=False, timeout=0.5, timeout_constrained=2, 
                  add_homing=False, post_optimize=False, max_solution_count=5)
    snode_schedule = ppline.tplan.get_best_schedule(at_home=False)
    snode_schedule += ppline.add_return_motion(snode_schedule[-1], initial_state=None, timeout=0.5, try_count=3)
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
    snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule, VEL, ACC)
    snode_schedule = snode_schedule_safe
    from_state = snode_schedule[-1].state
    snode_schedule_all.append(snode_schedule)

('track_face', 'track_face', 0, 0, 0)
generate table - PlanConditions
generate table - PlanList
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 0: required answers acquired  (20.6/100.0) ===============================
========================== FINISHED (20.6 / 100.0 s) ==============================]
Goal reached
Goal reached
generate table - Geometry
generate table - Object
generate table - PlanConditions
generate table - PlanList
generate table - Geometry
generate table - Object
generate table - PlanConditions
generate table - PlanList
generate table - Geometry
generate table - Object
generate table - PlanConditions
generate table - PlanList
generate table - Geometry
generate table - Object
generate table - PlanConditions
generate table - PlanList
generate table - Geometry
generate table - Object
generate table - PlanConditions
generate table - PlanList
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached


In [31]:
print(gtimer)

search_loop: 	147341.2 ms/3 = 49113.7 ms (20554.943/68947.71)
initialize_memory: 	8.8 ms/3 = 2.9 ms (1.135/5.13)
init_search: 	5.7 ms/3 = 1.9 ms (1.362/2.923)
test_connection: 	135094.8 ms/3023 = 44.7 ms (2.508/604.927)



## mix full schedule

In [34]:
gtimer.reset()
snode_schedule_cat = [snode_schedule_all[0][0]]
for snode_schedule in snode_schedule_all:
    snode_schedule_cat += snode_schedule[1:]

safe_mixed = snode_schedule_cat

### play schedule

In [35]:
ppline.play_schedule(safe_mixed, period=0.0002)

('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
('track_face', 'grip1', 1, 0, 0)->('track_face', 'grip1', 2, 0, 0)
('track_face', 'grip1', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
('track_face', 'grip1', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0)
('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0)
('grip1', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 1, 0)
('grip1', 'track_

### Run fully mixed schedule

In [36]:
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)

connection command:
indy0: True
panda1: True
rospy.init_node() has already been called with different arguments: ('task_planner', ['/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '-f', '/home/rnb/.local/share/jupyter/runtime/kernel-30887567-6330-48c2-b6af-4f3d8ab394d6.json'], True, None, False, True)


### Check gripper function
* **[IMPORTANT]** Check if panda gripper is open after below cell

In [45]:
crob.grasp(False, True)
time.sleep(0.5)
crob.grasp(False, False)

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)


In [46]:
crob.joint_move_make_sure(crob.home_pose)

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)


In [47]:
indy = crob.robot_dict["indy0"]
brush_face = pscene.actor_dict['brush_face']

In [48]:
f_switch = ForceModeSwitcher(pscene, mplan, brush_face, 
                             VEL=0.2, ACC=0.2)

In [49]:
ppline.execute_schedule(safe_mixed, 
                        mode_switcher=f_switch,
                        multiproc=False, error_stop_deg=None, one_by_one=True)

Connect: Server IP (192.168.21.6)
http://192.168.21.6:9990/param_setting?switch_control0=1
http://192.168.21.6:9990/param_setting?switch_control0=2
http://192.168.21.6:9990/param_setting?switch_control0=1
Connect: Server IP (192.168.21.6)
http://192.168.21.6:9990/param_setting?switch_control0=0
Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
No JSON object could be decoded


KeyError: 'qcount'

## 이슈 발생
* switch할 때 reset 안하고 push(Q) 하기 때문에 온라인 모드 동작이 되었음 --> 불안정